In [ ]:
from manim import *
import textwrap
import numpy as np

class Manim3DDeepdive(ThreeDScene):
    def construct(self):
        # ==========================================
        # 0. CONFIGURATION & UI SETUP
        # ==========================================
        # World Origin: Shifted LEFT to make room for UI
        self.axes_origin = ORIGIN + LEFT * 3.5
        # Camera Focus: Shifted to match the World Origin
        self.frame_shift = self.axes_origin 
        self.camera_zoom = 0.65
        
        # UI Position (Far Left)
        self.ui_bg_pos = LEFT * 6.2

        # Initialize UI (Background, Code Box, Description Box)
        self.setup_fixed_ui()
        
        # Set Initial Camera (Standard 3D View)
        self.set_camera_orientation(
            phi=70 * DEGREES,      
            theta=-30 * DEGREES,   
            zoom=self.camera_zoom,
            frame_center=self.frame_shift
        )

        self.set_description("Welcome. We have shifted the 3D origin\nto the side to allow for this UI overlay.")
        self.wait(2)

        # ==========================================
        # 1. AXES & VECTORS
        # ==========================================
        self.run_axes_setup()

        # ==========================================
        # 2. CAMERA MECHANICS
        # ==========================================
        self.run_camera_deep_dive()

        # ==========================================
        # 3. COORDINATE LOGIC (Local vs Global)
        # ==========================================
        self.run_positioning_deep_dive()

        # ==========================================
        # 4. ROTATION & PIVOTS
        # ==========================================
        self.run_cube_demo()

        # ==========================================
        # 5. TRANSFORMATIONS (Folding 2D to 3D)
        # ==========================================
        self.run_transformation_demo()

        # ==========================================
        # 6. SURFACES & VECTOR FIELDS
        # ==========================================
        self.run_surface_and_field_demo()

        # ==========================================
        # 7. PATHS & SHADING
        # ==========================================
        self.run_path_and_shading_demo()

        # ==========================================
        # 8. BILLBOARDING (FIXED)
        # ==========================================
        self.run_billboarding_demo()

        # End
        self.set_description("Tutorial Complete.\nBeginning ambient rotation.")
        self.update_code("self.begin_ambient_camera_rotation()")
        self.begin_ambient_camera_rotation(rate=0.15)
        self.wait(5)

    # ---------------------------------------------------------
    # UI & HELPER METHODS
    # ---------------------------------------------------------
    def setup_fixed_ui(self):
        self.ui_bg = Rectangle(
            width=5.5, height=10, 
            color=BLACK, fill_color=BLACK, fill_opacity=1, stroke_width=0
        )
        self.ui_bg.move_to(self.ui_bg_pos)
        self.ui_bg.set_z_index(10)
        
        self.title = Text("Manim 3d Axis, Camera, Position, Transform, Surface And Path", font_size=28, weight=BOLD)
        self.title.to_corner(UL, buff=0.5).shift(LEFT * 4)
        self.title.set_z_index(12)

        self.desc_pos = self.title.get_bottom() + DOWN * 1.0

        self.code_border = Rectangle(
            width=5.0, height=5.0, color=GRAY, stroke_width=2
        )
        self.code_border.move_to(self.ui_bg_pos + DOWN * 2)
        self.code_border.set_z_index(11)
        
        self.t_code_header = Text("Implementation:", font_size=16, color=BLUE)
        self.t_code_header.next_to(self.code_border, UP, aligned_edge=LEFT)
        self.t_code_header.set_z_index(12)
        
        self.add_fixed_in_frame_mobjects(
            self.ui_bg, self.code_border, self.title, self.t_code_header
        )

    def set_description(self, text):
        if hasattr(self, 'current_desc'):
            self.remove_fixed_in_frame_mobjects(self.current_desc)
            self.remove(self.current_desc)
        
        self.current_desc = Text(
            textwrap.dedent(text), 
            font_size=18, 
            color=YELLOW, 
            line_spacing=1.2,
            font="Sans"
        )
        self.current_desc.move_to(self.desc_pos, aligned_edge=UP)
        self.current_desc.set_z_index(12)
        self.add_fixed_in_frame_mobjects(self.current_desc)

    def update_code(self, code_text):
        if hasattr(self, 'code_group'):
            self.remove_fixed_in_frame_mobjects(self.code_group)
            self.remove(self.code_group)
            
        lines = textwrap.dedent(code_text).strip().split('\n')
        self.code_group = VGroup()
        
        start_point = self.code_border.get_top() + DOWN * 0.5 + LEFT * 2.2
        
        for i, line in enumerate(lines):
            c = WHITE
            s = line.strip()
            if s.startswith("#"): c = GREY
            elif "class" in s or "def" in s or "lambda" in s: c = ORANGE
            elif "move_camera" in s or "play" in s or "animate" in s: c = BLUE_B
            elif "phi" in s or "theta" in s or "axis" in s: c = YELLOW
            elif "UP" in s or "RIGHT" in s or "OUT" in s: c = RED
            elif "Surface" in s or "Cube" in s or "Arrow" in s: c = GREEN
            
            t = Text(line, font="Monospace", font_size=14, color=c)
            t.move_to(start_point + DOWN * (i * 0.35), aligned_edge=LEFT)
            self.code_group.add(t)
            
        self.code_group.set_z_index(13)
        self.add_fixed_in_frame_mobjects(self.code_group)

    # ---------------------------------------------------------
    # CHAPTER 1: AXES
    # ---------------------------------------------------------
    def run_axes_setup(self):
        self.set_description("1. The Framework\nWe place axes at our shifted origin.\nRed=X, Green=Y, Blue=Z (Right-Hand Rule).")
        self.update_code("""
        axes = ThreeDAxes()
        axes.move_to(self.axes_origin)
        self.add(axes)
        """)
        
        self.axes = ThreeDAxes(x_length=6, y_length=6, z_length=5)
        self.axes.move_to(self.axes_origin)
        
        x_lbl = self.axes.get_x_axis_label("X").set_color(RED)
        y_lbl = self.axes.get_y_axis_label("Y").set_color(GREEN)
        z_lbl = self.axes.get_z_axis_label("Z").set_color(BLUE)
        self.axis_labels = VGroup(x_lbl, y_lbl, z_lbl)

        self.play(Create(self.axes), FadeIn(self.axis_labels))
        
        i_vec = Arrow(self.axes.c2p(0,0,0), self.axes.c2p(1,0,0), color=RED, buff=0)
        j_vec = Arrow(self.axes.c2p(0,0,0), self.axes.c2p(0,1,0), color=GREEN, buff=0)
        k_vec = Arrow(self.axes.c2p(0,0,0), self.axes.c2p(0,0,1), color=BLUE, buff=0)

        self.update_code("""
        # Unit Vectors
        i_vec = Arrow(
            self.axes.c2p(0,0,0), 
            self.axes.c2p(1,0,0), 
            color=RED, 
            buff=0
        )
        j_vec = ...
        k_vec = ...
                         
        self.play(GrowArrow(i_vec), ...)
        """)        
        
        self.set_description("Unit Vectors:\ni (X), j (Y), k (Z)")
        self.play(GrowArrow(i_vec), GrowArrow(j_vec), GrowArrow(k_vec))
        self.wait(1)
        self.play(FadeOut(i_vec), FadeOut(j_vec), FadeOut(k_vec))

    # ---------------------------------------------------------
    # CHAPTER 2: CAMERA
    # ---------------------------------------------------------
    def run_camera_deep_dive(self):
        ref_obj = Sphere(radius=1.0).set_fill(BLUE, 0.5).move_to(self.axes_origin)
        self.play(FadeIn(ref_obj))

        self.set_description("2A. Theta (Azimuth)\nRotating around the Z-axis (Pan).\n-90 deg is 'Right', -180 is 'Back'.")
        self.update_code(
            """
            self.move_camera(
                theta=-120*DEGREES,
                run_time=2
            )"""
        )
        self.move_camera(theta=-120*DEGREES, run_time=2)
        
        self.set_description("2B. Phi (Polar/Zenith)\nTilting down from the Z-axis.\n0 deg = Top Down. 90 deg = Side.")
        self.update_code(
            """
            self.move_camera(
                phi=30*DEGREES,
                run_time=2
            )"""
        )
        self.move_camera(phi=30*DEGREES, run_time=2)
        
        self.set_description("2C. Zoom (Dolly)\nMoving the camera physically closer.")
        self.update_code(
            """
            self.move_camera(
                zoom=1.0,
                run_time=2
            )"""
        )
        self.move_camera(zoom=1.0, run_time=2)
        
        self.set_description("Resetting Camera...")
        self.update_code("# Resetting angles")
        self.move_camera(
            phi=70*DEGREES, theta=-30*DEGREES, zoom=self.camera_zoom,
            frame_center=self.frame_shift, run_time=1.5
        )
        self.play(FadeOut(ref_obj))

    # ---------------------------------------------------------
    # CHAPTER 3: POSITIONING
    # ---------------------------------------------------------
    def run_positioning_deep_dive(self):
        self.set_description("3. Coordinates: Global vs Local\nBecause our axes are shifted, we MUST use\naxes.c2p(x,y,z) to plot correctly.")
        self.update_code("""
        # WRONG: dot.move_to(RIGHT*2)
        
        # CORRECT:
        dot = Sphere(radius=0.15, color=YELLOW)
        target = axes.c2p(2, 2, 2)
                         
        line = Line(axes.c2p(0,0,0), target, color=YELLOW_E)
        self.play(Create(line), dot.animate.move_to(target))
        """)
        
        dot = Sphere(radius=0.15, color=YELLOW).move_to(self.axes_origin)
        self.play(FadeIn(dot))
        
        target = self.axes.c2p(2, 2, 2)
        line = Line(self.axes.c2p(0,0,0), target, color=YELLOW_E)
        
        self.play(Create(line), dot.animate.move_to(target))
        self.wait()
        self.play(FadeOut(dot), FadeOut(line))

    # ---------------------------------------------------------
    # CHAPTER 4: ROTATION
    # ---------------------------------------------------------
    def run_cube_demo(self):
        self.set_description("4. Rotation Pivots\nRotations happen around the object's center\nunless 'about_point' is specified.")
        self.update_code("""
        # Local Rotation (Spins in place)
        cube = Cube(side_length=2, ...)                 
        Rotate(cube, angle=PI/2, axis=UP)
        """)
        
        cube = Cube(side_length=2, fill_opacity=0.5, stroke_width=2, fill_color=PURPLE)
        cube.move_to(self.axes_origin) 
        
        self.play(FadeIn(cube))
        self.play(Rotate(cube, angle=PI/2, axis=UP), run_time=1.5)
        
        self.set_description("4B. Orbiting\nTo orbit, we rotate 'about_point=ORIGIN'.")
        self.update_code("""
        Rotate(
            cube, 
            angle=PI, 
            axis=OUT
            about_point=axes_origin
        )
        """)
        self.play(cube.animate.move_to(self.axes.c2p(2,0,0)))
        self.play(Rotate(cube, angle=PI, axis=OUT, about_point=self.axes_origin), run_time=2)
        
        self.play(FadeOut(cube))

    # ---------------------------------------------------------
    # CHAPTER 5: TRANSFORMATIONS
    # ---------------------------------------------------------
    def run_transformation_demo(self):
        self.set_description("5. Transformation (Folding)\nWe can apply a function to 'fold' a 2D plane\ninto a 3D shape (e.g., Cylinder).")
        self.update_code("""
        def fold_func(p):
            x, y, z = p
            return axes.c2p(x, sin(y), cos(y))
            
        plane.apply_function(fold_func)
        """)
        
        grid = NumberPlane(
            x_range=[-2, 2], y_range=[-PI, PI], 
            background_line_style={"stroke_color": TEAL, "stroke_width": 2}
        )
        grid.move_to(self.axes_origin)
        self.play(Create(grid))
        self.wait(0.5)

        def folding_logic(point):
            local_p = point - self.axes_origin
            x, y, z = local_p
            return self.axes_origin + np.array([x, 1.5*np.sin(y), 1.5*np.cos(y)])

        self.play(grid.animate.apply_function(folding_logic), run_time=3)
        self.wait(1)
        self.play(FadeOut(grid))

    # ---------------------------------------------------------
    # CHAPTER 6: SURFACES & VECTOR FIELDS
    # ---------------------------------------------------------
    def run_surface_and_field_demo(self):
        self.set_description("6. Parametric Surface\nDefining (u,v) ranges to draw 3D waves.")
        self.update_code("""
        Surface(
           lambda u, v: axes.c2p(
               u, v, sin(u)*cos(v),
           ),
            u_range=[-2, 2], v_range=[-2, 2],
            resolution=(16, 16),
            fill_opacity=0.6,
            checkerboard_colors=[TEAL, TEAL_E]
        )
        """)
        
        surface = Surface(
            lambda u, v: self.axes.c2p(u, v, 0.8 * np.sin(1.5*u) * np.cos(1.5*v)),
            u_range=[-2, 2], v_range=[-2, 2],
            resolution=(16, 16),
            fill_opacity=0.6,
            checkerboard_colors=[TEAL, TEAL_E]
        )
        self.play(Create(surface), run_time=2)
        
        self.set_description("6B. 3D Vector Fields\nVisualizing forces in 3D space.")
        self.update_code("""
        def func(pos):
             return np.array([-y, x, 0.2])
             
        ArrowVectorField(
            func,
            x_range=[..., ...],
            y_range=[..., ...],
            z_range=[..., ...],
            colors=[BLUE_E, GREEN_E],
        )
        """)
        self.play(FadeOut(surface))

        def field_func(pos):
            x, y, z = self.axes.p2c(pos) 
            return np.array([-y*0.5, x*0.5, 0.5])

        field = ArrowVectorField(
            field_func,
            x_range=[self.axes_origin[0]-2, self.axes_origin[0]+2, 1],
            y_range=[self.axes_origin[1]-2, self.axes_origin[1]+2, 1],
            z_range=[self.axes_origin[2]-1, self.axes_origin[2]+1, 1],
            colors=[BLUE_E, GREEN_E],
        )
        self.play(Create(field))
        self.wait(2)
        self.play(FadeOut(field))

    # ---------------------------------------------------------
    # CHAPTER 7: PATHS & SHADING
    # ---------------------------------------------------------
    def run_path_and_shading_demo(self):
        self.set_description("7. Paths & Shading\nCalculating shading adds depth perception\nbased on surface normals.")
        self.update_code("""
        # Path
        curve = ParametricFunction(
            lambda t: self.axes.c2p(2*np.cos(t), 2*np.sin(t), t/2),
            t_range=[-2*PI, 2*PI], color=ORANGE
        )
                         
        MoveAlongPath(dot, curve, run_time=3, rate_func=linear)
        
        # Shading
        sphere.set_shade_in_3d(True)
        """)
        
        curve = ParametricFunction(
            lambda t: self.axes.c2p(2*np.cos(t), 2*np.sin(t), t/2),
            t_range=[-2*PI, 2*PI], color=ORANGE
        )
        dot = Sphere(radius=0.2, color=RED).move_to(curve.get_start())
        
        self.play(Create(curve), FadeIn(dot))
        self.play(MoveAlongPath(dot, curve), run_time=3, rate_func=linear)
        self.play(FadeOut(curve), FadeOut(dot))
        
        shaded_sphere = Sphere(radius=1.5).move_to(self.axes_origin)
        shaded_sphere.set_shade_in_3d(True) 
        shaded_sphere.set_fill(color=BLUE)
        
        self.play(GrowFromCenter(shaded_sphere))
        self.play(Rotate(shaded_sphere, angle=PI, axis=UP), run_time=2)
        self.play(FadeOut(shaded_sphere))

    # ---------------------------------------------------------
    # CHAPTER 8: BILLBOARDING (RE-FIXED)
    # ---------------------------------------------------------
    def run_billboarding_demo(self):
        self.set_description("8. Billboarding\nUsing an 'updater' to force text to\nalways look at the camera.")
        self.update_code("""
        label = Text("Front Face", ...)
                         
        def update_label(m):
           phi = camera.get_phi()
           theta = camera.get_theta()
           m.become(original)
           m.rotate(theta + PI/2, axis=OUT)
           m.rotate(phi, axis=RIGHT)
        
        label.add_updater(update_label)
        """)
        
        # 1. Create label (starts flat on XY plane)
        label = Text("Front Face", color=YELLOW, font_size=36)
        label.move_to(self.axes_origin + UP*1.5)
        
        # 2. Create a copy to act as the "reset" state for the updater
        label_original = label.copy()
        
        # 3. Define the Billboarding Updater
        def billboard_updater(mob):
            # Reset to original state (flat)
            mob.become(label_original)
            
            # Get camera angles
            phi = self.camera.get_phi()
            theta = self.camera.get_theta()
            
            # Rotate to face the camera
            # A) Rotate around Z axis to face the azimuth
            mob.rotate(theta + PI/2, axis=OUT)
            # B) Rotate around RIGHT axis to face the elevation
            mob.rotate(phi, axis=RIGHT)

        label.add_updater(billboard_updater)
        self.add(label)
        self.play(Write(label))
        
        # 4. Move Camera to prove it works
        self.move_camera(theta=-10 * DEGREES, run_time=2)
        self.move_camera(theta=-50 * DEGREES, run_time=2)
        
        label.clear_updaters()
        self.play(FadeOut(label))

%manim -ql -v warning Manim3DDeepdive